In [93]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import itertools

# Setting data

In [94]:

df = pd.read_excel('basket_abcde.xlsx')
df_graph = pd.read_excel('basket_abcde.xlsx')

In [95]:
df = df.rename(columns={'Unnamed: 0':'invoiceid'})
df = df.set_index('invoiceid')

In [96]:
item_list = list(df.columns)
print(item_list)

['a', 'b', 'c', 'd', 'e']


In [97]:
df_graph = df_graph.rename(columns={'Unnamed: 0':'invoiceid'})
df_graph = df_graph.set_index('invoiceid')

In [98]:
df['a'] = df['a']*2
df['b'] = df['b']*3
df['c'] = df['c']*1
df['d'] = df['d']*2
df['e'] = df['e']*4


In [61]:
#df

# Graph

## Finding subgraphs using the binary valued basket

In [35]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [63]:
#df_graph

In [99]:
def adjacency_from_dataframe(df):
    a = df.to_numpy()
    m = a.shape[1]
    adj = np.empty((m, m), dtype=np.bool)
    for j in range(m):
        col = a[:, j]
        rows = a[col == 1]
        adj[j] = np.logical_or.reduce(rows)
    return adj

In [102]:
items = item_list
adj = adjacency_from_dataframe(df_graph)

g = nx.from_numpy_matrix(adj)

In [103]:
g_df = nx.to_pandas_adjacency(g)
g_df

,0,1,2,3,4
0,1.0,1.0,1.0,0.0,0.0
1,1.0,1.0,1.0,0.0,0.0
2,1.0,1.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,1.0
4,0.0,0.0,0.0,1.0,1.0


In [104]:
components = [g.subgraph(c).copy() for c in nx.connected_components(g)]
for idx,g in enumerate(components,start=1):
    print(f"Component {idx}: Nodes: {g.nodes()} Edges: {g.edges()}")
    

Component 1: Nodes: [0, 1, 2] Edges: [(0, 0), (0, 1), (0, 2), (1, 1), (1, 2), (2, 2)]
Component 2: Nodes: [3, 4] Edges: [(3, 3), (3, 4), (4, 4)]


In [77]:
list(components[0].edges())[0]

list(g.edges())[0]

0

In [105]:
for g in components:
    print("subgraph")
    for edge in list(g.edges()):
        if(item_list[edge[0]]!= item_list[edge[1]]):
            print(item_list[edge[0]], item_list[edge[1]])

subgraph
a b
a c
b c
subgraph
d e


In [113]:
item_list.sort()
item_list

['a', 'b', 'c', 'd', 'e']

# Characteristic function

## 1. Subset creation
## 2. Characteristic function calculation for each subset

In [109]:
# Finding invoice wise sum

basket = df
invoice_sum = list(basket.sum(axis = 1))
#invoice_sum

In [158]:
basket[basket!=0].count()

a    10
b    24
c    10
d    17
e    12
dtype: int64

In [111]:
print('itemwise sum: \n',basket.sum())
print('total sum: ', basket.sum().sum())

itemwise sum: 
 a    20
b    72
c    10
d    34
e    48
dtype: int64
total sum:  184


In [116]:
# Creating subsets

import itertools
subsets = []
stuff = item_list
for L in range(0, len(stuff)+1):
    for subset in itertools.combinations(stuff, L):
        subsets.append(list(subset))

#subsets

In [118]:
# finding the row wise items (have to optimise the step)

col_list_full = []
for index, row in basket.iterrows():
    col_list = []
    for col in basket:
            #if col in 
        if(basket.at[index,col]!=0.0):
            col_list.append(col)
    col_list_full.append(col_list)
    #print('-------------------------------------------------------------------------')
            #print(round(row.sum(),2))
#col_list_full   

In [119]:
# grouping the invoice and columns based on the subsets

before_grp = []
for i in range (len(basket)):
    temp = [str(col_list_full[i]),invoice_sum[i]]
    #print(col_list_full[i], ' ',invoice_sum[i])
    before_grp.append(temp)

before_grp

e_dataframe = pd.DataFrame(before_grp)
e_dataframe.columns = ['a', 'b']

e_dataframe

groupped_sum = pd.DataFrame(e_dataframe.groupby(['a']).sum())
groupped_sum.reset_index(level=0, inplace=True)
groupped_sum.columns = ['a', 'b']

In [120]:
# finding characteristic function value for each group and assigning them to subset list
all_sums = []
for sub in subsets:
    sum1 = 0
    print('subset id:', subsets.index(sub))
    print("step1")
    #print(sub)
    print('------------')
    stuff = sub
    new_sub = []
    for L in range(1, len(stuff)+1):
        for subset in itertools.combinations(stuff, L):
            new_sub.append(list(subset))
    print(new_sub)
    for item in new_sub:
        index = groupped_sum[groupped_sum['a'] == str(item)].index
        if index.empty:
            print('None')
        else:
            print(item, index)
            sum1+=groupped_sum['b'][index].values[0]
    all_sums.append(sum1)
    print('------------------------------------------------------------')

subset id: 0
step1
------------
[]
------------------------------------------------------------
subset id: 1
step1
------------
[['a']]
None
------------------------------------------------------------
subset id: 2
step1
------------
[['b']]
['b'] Int64Index([3], dtype='int64')
------------------------------------------------------------
subset id: 3
step1
------------
[['c']]
['c'] Int64Index([4], dtype='int64')
------------------------------------------------------------
subset id: 4
step1
------------
[['d']]
['d'] Int64Index([6], dtype='int64')
------------------------------------------------------------
subset id: 5
step1
------------
[['e']]
['e'] Int64Index([7], dtype='int64')
------------------------------------------------------------
subset id: 6
step1
------------
[['a'], ['b'], ['a', 'b']]
None
['b'] Int64Index([3], dtype='int64')
['a', 'b'] Int64Index([1], dtype='int64')
------------------------------------------------------------
subset id: 7
step1
------------
[['a'], ['

In [121]:
all_sums

[0,
 0,
 54,
 3,
 14,
 8,
 69,
 15,
 14,
 8,
 57,
 68,
 62,
 17,
 11,
 82,
 102,
 83,
 77,
 29,
 23,
 82,
 71,
 65,
 136,
 85,
 116,
 110,
 151,
 97,
 139,
 184]

In [123]:
new_dict = {}
for i in range(len(all_sums)):
    new_dict.update({str(subsets[i]) : all_sums[i] })
    #print(subsets[i], " ", all_sums[i])

In [126]:
#new_dict

# Finding marginal Contribution

In [127]:
dict1 = {'subsets': subsets}
subsets_df = pd.DataFrame((dict1))
subsets_df

,subsets
0,[]
1,[a]
2,[b]
3,[c]
4,[d]
5,[e]
6,"[a, b]"
7,"[a, c]"
8,"[a, d]"
9,"[a, e]"


In [129]:
# adding items to the columns
struct1 = pd.concat([subsets_df,pd.DataFrame(columns=item_list)])
struct1

,subsets,a,b,c,d,e
0,[],NaN,NaN,NaN,NaN,NaN
1,[a],NaN,NaN,NaN,NaN,NaN
2,[b],NaN,NaN,NaN,NaN,NaN
3,[c],NaN,NaN,NaN,NaN,NaN
4,[d],NaN,NaN,NaN,NaN,NaN
5,[e],NaN,NaN,NaN,NaN,NaN
6,"[a, b]",NaN,NaN,NaN,NaN,NaN
7,"[a, c]",NaN,NaN,NaN,NaN,NaN
8,"[a, d]",NaN,NaN,NaN,NaN,NaN
9,"[a, e]",NaN,NaN,NaN,NaN,NaN


In [132]:
columns_list = item_list
#columns_list

index_list = subsets
#index_list

In [151]:

for index in index_list:
    print('========================================================================')
    print('item coalition (index): ', index)
    print('-----------------------')
    for col in columns_list:
        
        print('item (column) = ', col)
        if col in index:
            temp = index[:]
            #temp.append(col)
            #if (index == list(str(col))):
            #    print('marginal contribution = v()', col)
            #else:
            #    temp.remove(col)
            #    print('marginal contribution = v()', index, ' - v() ', temp)
            #    #print('item list = ', index, ' value of items ', temp)
            #    #continue
            #
            temp.remove(col)
            print('marginal contribution = v()', index, ' - v() ', temp)
            #print('marginal contribution = v()', new_dict[str(index)], ' - v() ',new_dict[str(temp)])
            struct1.loc[subsets.index(index),col] = new_dict[str(index)] - new_dict[str(temp)]
            #print('item list = ', index, ' value of items ', temp)
            print('-----------------------')
            #print(index, list(str(col)))
        else:
            print('marginal contribution = 0')
            print('-----------------------')
            continue
        
        
        
            #temp.append(0)
    
    #print(temp)
    #new_struct.append(temp)
#

#pd.DataFrame(new_struct)

item coalition (index):  []
-----------------------
item (column) =  a
marginal contribution = 0
-----------------------
item (column) =  b
marginal contribution = 0
-----------------------
item (column) =  c
marginal contribution = 0
-----------------------
item (column) =  d
marginal contribution = 0
-----------------------
item (column) =  e
marginal contribution = 0
-----------------------
item coalition (index):  ['a']
-----------------------
item (column) =  a
marginal contribution = v() ['a']  - v()  []
-----------------------
item (column) =  b
marginal contribution = 0
-----------------------
item (column) =  c
marginal contribution = 0
-----------------------
item (column) =  d
marginal contribution = 0
-----------------------
item (column) =  e
marginal contribution = 0
-----------------------
item coalition (index):  ['b']
-----------------------
item (column) =  a
marginal contribution = 0
-----------------------
item (column) =  b
marginal contribution = v() ['b']  - v() 

In [150]:
col = 'e'
idx = ['a', 'b']
subsets.index(idx)
struct1.loc[subsets.index(idx),col]

nan

In [152]:
struct1

,subsets,a,b,c,d,e
0,[],NaN,NaN,NaN,NaN,NaN
1,[a],0,NaN,NaN,NaN,NaN
2,[b],NaN,54,NaN,NaN,NaN
3,[c],NaN,NaN,3,NaN,NaN
4,[d],NaN,NaN,NaN,14,NaN
5,[e],NaN,NaN,NaN,NaN,8
6,"[a, b]",15,69,NaN,NaN,NaN
7,"[a, c]",12,NaN,15,NaN,NaN
8,"[a, d]",0,NaN,NaN,14,NaN
9,"[a, e]",0,NaN,NaN,NaN,8


In [159]:
struct1.mean()/basket[basket!=0].count()

a    1.800000
b    2.750000
c    1.350000
d    2.588235
e    3.166667
dtype: float64

6